In [17]:
import pandas as pd
import csv
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import *
from tensorflow.keras.layers import *

In [18]:
import os
import numpy as np
import contractions as ctc
import re

In [19]:
from tensorflow.keras import models,layers
import tensorflow.keras.backend as K
from matplotlib import pyplot as plt
from tensorflow.keras.models import load_model

In [168]:
from sklearn.metrics import classification_report,f1_score

# Load BBC dataset and preprocess

In [4]:
df_train=pd.read_csv("./TrainData.csv")

In [5]:
labels={'business':0, 'tech':1, 'politics':2, 'sport':3, 'entertainment':4}

In [6]:
df_train["labels"]=df_train["Category"].map(labels)

In [7]:
df_train.head()

,Text,Category,labels
0,worldcom ex-boss launches defence lawyers defe...,business,0
1,german business confidence slides german busin...,business,0
2,bbc poll indicates economic gloom citizens in ...,business,0
3,lifestyle governs mobile choice faster bett...,tech,1
4,enron bosses in $168m payout eighteen former e...,business,0


In [9]:
df_train.Text[0]

'worldcom ex-boss launches defence lawyers defending former worldcom chief bernie ebbers against a battery of fraud charges have called a company whistleblower as their first witness.  cynthia cooper  worldcom s ex-head of internal accounting  alerted directors to irregular accounting practices at the us telecoms giant in 2002. her warnings led to the collapse of the firm following the discovery of an $11bn (£5.7bn) accounting fraud. mr ebbers has pleaded not guilty to charges of fraud and conspiracy.  prosecution lawyers have argued that mr ebbers orchestrated a series of accounting tricks at worldcom  ordering employees to hide expenses and inflate revenues to meet wall street earnings estimates. but ms cooper  who now runs her own consulting business  told a jury in new york on wednesday that external auditors arthur andersen had approved worldcom s accounting in early 2001 and 2002. she said andersen had given a  green light  to the procedures and practices used by worldcom. mr ebb

In [11]:
## Putting space b/w Special Characters
def is_special(text):
    rem = ''
    for i in text: 
        if i.isalnum():
            rem = rem + i
        else:
            rem = rem + ' '
            rem=rem+i
            rem = rem + ' '
    return rem

In [12]:
## removing extra spsces from text
def rem_extra(text):
    rem=re.sub(' +', ' ',text)
    return str(rem).strip()

In [25]:
df_train.Text=df_train.Text.apply(is_special)

In [26]:
df_train.Text=df_train.Text.apply(rem_extra)

# Loading Pretrained GloVe

In [21]:
embeddings_index = dict()
addrs="../ADL1/embeddings/glove.6B.300d.txt"
f = open(addrs)
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 400000 word vectors.


In [99]:
x_train=np.zeros((len(df_train.Text),300))
k=0;
for i in df_train.Text:
    vec=np.zeros((300))
    c=0;
    for word in i.split(): 
        embedding_vector = embeddings_index.get(word)
        #print(embedding_vector.shape)
        if embedding_vector is not None:
            vec+=embedding_vector
            c+=1
        else:
            print(word)
            pass
    x_train[k]=vec/c
    k+=1
    

11bn
7bn
ebber
75bn
weidensteiner
phonecam
168m
168m
89m
168m
155m
2001alleging
145m
truanted
785m
4bn
784m
374m
6bn
506m
4bn
142bn
5bn
475m
daimerchrysler
gigajoule
gigajoule
7bn
5bn
548p
65bn
97bn
89bn
macphisto
likhotseva
hadn
v603sh
afficiandos
s6500
cazalets
2mbps
2mbps
40mbps
40mbps
6bn
renationalise
6bn
6bn
oganesyan
27bn
lukashov
lukashov
kalanick
commoditising
overpeer
broes
broes
6bn
6bn
16bn
lifeplus
magnenat
miralab
lifeplus
2d3
2d3
thoday
thoday
thoday
slushier
nm2
nm2
nm2
stollenmayer
wyver
wyver
wyver
nm2
wraz
trivialises
wraz
wraz
ruigang
gescher
gescher
gamesindustry
bridgewell
160gb
20gb
160gb
tveter
westfallen
68m
89m
47m
halser
shadowcrew
tyukanov
shadowcrew
carderplanet
darkprofits
nhtcu
shadowcrew
shadowcrew
nhtcu
ofex
pitylak
oftexas
payperaction
leadplex
leadplex
pitylak
leadplex
payperaction
pitylak
islero
moblogs
fonetography
ilston
220m
117m
648m
tsusaka
25lb
naftohaz
morquendi
morquendi
540bn
77bn
472bn
islamaphobia
jammins
fazelas
fazelas
173m
syndol
meltus

In [42]:
x_train.shape

(1490, 300)

# Load and preprocess test data

In [68]:
df_test=pd.read_csv("./TestData_Inputs.csv")

In [69]:
df_test.head()

,ArticleId,Text
0,1018,qpr keeper day heads for preston queens park r...
1,1319,software watching while you work software that...
2,1138,d arcy injury adds to ireland woe gordon d arc...
3,459,india s reliance family feud heats up the ongo...
4,1020,boro suffer morrison injury blow middlesbrough...


In [70]:
df_test.Text=df_test.Text.apply(is_special)

In [71]:
df_test.Text=df_test.Text.apply(rem_extra)

In [79]:
x_test=np.zeros((len(df_test.Text),300))
k=0;
for i in df_test.Text:
    vec=np.zeros((300))
    c=0;
    for word in i.split(): 
        embedding_vector = embeddings_index.get(word)
        #print(embedding_vector.shape)
        if embedding_vector is not None:
            vec+=embedding_vector
            c+=1
        else:
            print(word)
            pass
    x_test[k]=vec/c
    k+=1
    

3ami
nhtcu
3ami
ellsmore
3ami
17bn
745bn
dhiru
35bn
4bn
50bn
8bn
deselection
25bn
135m
slba
18mbps
1mbps
2mbps
gmipoll
leoncia
77m
unfeasibility
chadti
427bn
230bn
80bn
23bn
6bn
5bn
200bn
hadn
brownite
parbutt
227bn
331bn
14bn
27bn
14bn
4bn
20bn
mitroshin
surgener
lakomskis
lakomskis
waasdorp
klerck
surgener
petruzzella
osentowski
x505
ux50
300d
novatech
515m
515m
277m
375m
bacap
47bn
tambone
intoppa
legisation
politcians
brownbeck
weblogger
sigarchi
sigarchi
motjaba
saminejad
sigarchi
weblogger
sigarchi
sigarchi
sigarchi
sigarchi
sigarchi
sigarchi
sentencers
sentencers
masscre
enfish
chandratillake
chandratillake
computerandvideogames
gadaravice
krivorizhstal
424m
reisurance
goetti
mediterranne
floiran
woest
analsyts
ripguard
ripguard
ripguard
ripguard
ripguard
ripguard
ripguard
4bn
6bn
centralises
tapwave
hellraisers
broadbanders
15mb
1mbps
mccririck
mccririck
mccririck
138m
oxby
wlts
wltm
socieded
astiazaran
astiazaran
confirned
activison
oudendijk
oudendijk
42in
106cm
oudendijk
blo

In [74]:
df_labels=pd.read_csv("./TestLabels.csv")

In [76]:
df_labels["labels"]=df_labels["Category"].map(labels)

In [77]:
df_labels.head()

,ArticleId,Category,labels
0,1018,sport,3
1,1319,tech,1
2,1138,sport,3
3,459,business,0
4,1020,sport,3


# DAN+DNN


In [113]:
model1=Sequential()
model1.add(InputLayer((300)))
model1.add(Dense(256))
model1.add(Dense(512))
model1.add(Dense(256,activation="relu"))
model1.add(Dense(128,activation="relu"))
model1.add(Dense(5,activation="softmax"))

In [114]:
model1.compile(loss="sparse_categorical_crossentropy",optimizer="adam",metrics="accuracy")

In [117]:
es=keras.callbacks.EarlyStopping(monitor='val_loss',patience=10,mode='min')

In [118]:
model1.fit(x_train,df_train.labels.values,epochs=100,batch_size=32,validation_split=0.1,callbacks=[es])

Epoch 1/100
42/42 [==============================] - 0s 4ms/step - loss: 0.1734 - accuracy: 0.9754 - val_loss: 0.6576 - val_accuracy: 0.9530
Epoch 2/100
42/42 [==============================] - 0s 3ms/step - loss: 0.0801 - accuracy: 0.9858 - val_loss: 0.5445 - val_accuracy: 0.9732
Epoch 3/100
42/42 [==============================] - 0s 3ms/step - loss: 0.0769 - accuracy: 0.9814 - val_loss: 0.7923 - val_accuracy: 0.9463
Epoch 4/100
42/42 [==============================] - 0s 3ms/step - loss: 0.0824 - accuracy: 0.9843 - val_loss: 0.5759 - val_accuracy: 0.9732
Epoch 5/100
42/42 [==============================] - 0s 4ms/step - loss: 0.1248 - accuracy: 0.9828 - val_loss: 0.7031 - val_accuracy: 0.9463
Epoch 6/100
42/42 [==============================] - 0s 3ms/step - loss: 0.0375 - accuracy: 0.9881 - val_loss: 0.6081 - val_accuracy: 0.9664
Epoch 7/100
42/42 [==============================] - 0s 3ms/step - loss: 0.0677 - accuracy: 0.9858 - val_loss: 0.4027 - val_accuracy: 0.9866
Epoch 8/100
4

In [119]:
model1.evaluate(x_test,df_labels.labels.values)

23/23 [==============================] - 0s 1ms/step - loss: 0.5211 - accuracy: 0.9660


[0.5211055278778076, 0.9659863710403442]

In [175]:
y_pred=np.argmax(model1.predict(x_test),axis=1)

In [176]:
pr=classification_report(df_labels.labels.values,y_pred)

In [177]:
print(pr)

              precision    recall  f1-score   support

           0       0.96      0.93      0.95       176
           1       0.97      0.98      0.97       135
           2       0.93      0.97      0.95       143
           3       1.00      0.99      0.99       168
           4       0.96      0.96      0.96       113

    accuracy                           0.97       735
   macro avg       0.96      0.97      0.97       735
weighted avg       0.97      0.97      0.97       735



In [178]:
f1_score(df_labels.labels.values,y_pred,average='macro')

0.9657131627572149

# Load Pretrained FastText

In [120]:
embeddings_index = dict()
addrs="../ADL1/embeddings/wiki-news-300d-1M-subword.vec"
f = open(addrs)
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 999995 word vectors.


In [124]:
x_train_f=np.zeros((len(df_train.Text),300))
k=0
c=0
for i in df_train.Text:
    vec=np.zeros((300))
    for word in i.split(): 
        embedding_vector = embeddings_index.get(word)
        #print(embedding_vector.shape)
        if embedding_vector is not None:
            vec+=embedding_vector
            c+=1
        else:
            #print(word)
            pass
    x_train_f[k]=vec/c
    k+=1

9221


In [125]:
x_test_f=np.zeros((len(df_test.Text),300))
k=0;
for i in df_test.Text:
    vec=np.zeros((300))
    c=0;
    for word in i.split(): 
        embedding_vector = embeddings_index.get(word)
        #print(embedding_vector.shape)
        if embedding_vector is not None:
            vec+=embedding_vector
            c+=1
        else:
            #print(word)
            pass
    x_test_f[k]=vec
    k+=1
    

# DAN+DNN

In [126]:
model2=Sequential()
model2.add(InputLayer((300)))
model2.add(Dense(256))
model2.add(Dense(512))
model2.add(Dense(256,activation="relu"))
model2.add(Dense(128,activation="relu"))
model2.add(Dense(5,activation="softmax"))

In [128]:
model2.compile(loss="sparse_categorical_crossentropy",optimizer="adam",metrics="accuracy")

In [129]:
es=keras.callbacks.EarlyStopping(monitor='val_loss',patience=10,mode='min')

In [130]:
model2.fit(x_train_f,df_train.labels.values,epochs=100,batch_size=32,validation_split=0.1,callbacks=[es])

Epoch 1/100
42/42 [==============================] - 1s 12ms/step - loss: 2.6953 - accuracy: 0.5720 - val_loss: 0.7380 - val_accuracy: 0.7383
Epoch 2/100
42/42 [==============================] - 0s 3ms/step - loss: 0.6403 - accuracy: 0.8009 - val_loss: 0.7357 - val_accuracy: 0.7919
Epoch 3/100
42/42 [==============================] - 0s 3ms/step - loss: 0.4050 - accuracy: 0.8606 - val_loss: 0.3793 - val_accuracy: 0.8591
Epoch 4/100
42/42 [==============================] - 0s 3ms/step - loss: 0.2209 - accuracy: 0.9262 - val_loss: 0.2810 - val_accuracy: 0.9060
Epoch 5/100
42/42 [==============================] - 0s 3ms/step - loss: 0.1614 - accuracy: 0.9485 - val_loss: 0.3800 - val_accuracy: 0.8658
Epoch 6/100
42/42 [==============================] - 0s 4ms/step - loss: 0.1880 - accuracy: 0.9389 - val_loss: 0.3758 - val_accuracy: 0.8859
Epoch 7/100
42/42 [==============================] - 0s 3ms/step - loss: 0.1520 - accuracy: 0.9493 - val_loss: 0.3130 - val_accuracy: 0.9262
Epoch 8/100


In [131]:
model2.evaluate(x_test_f,df_labels.labels.values)

23/23 [==============================] - 0s 1ms/step - loss: 0.2627 - accuracy: 0.9605


[0.2626713812351227, 0.960544228553772]

In [171]:
y_pred=np.argmax(model2.predict(x_test_f),axis=1)

In [172]:
pr=classification_report(df_labels.labels.values,y_pred)

In [173]:
print(pr)

              precision    recall  f1-score   support

           0       0.96      0.91      0.94       176
           1       0.92      0.98      0.95       135
           2       0.95      0.98      0.96       143
           3       0.99      0.98      0.98       168
           4       0.97      0.96      0.97       113

    accuracy                           0.96       735
   macro avg       0.96      0.96      0.96       735
weighted avg       0.96      0.96      0.96       735



In [174]:
f1_score(df_labels.labels.values,y_pred,average='macro')

0.9608977969125032

# Load Pretrained word2vec

In [151]:
# create a weight matrix for words in training docs
from gensim import models

w2v = models.KeyedVectors.load_word2vec_format('../ADL1/embeddings/GoogleNews-vectors-negative300.bin', binary=True)



In [153]:
x_train_w2v=np.zeros((len(df_train.Text),300))
k=0;
for i in df_train.Text:
    vec=np.zeros((300))
    c=1;
    for word in i.split(): 
        try:
            vec+=w2v[word]
            c+=1
        except:
            #print(word)
            pass
    x_train_w2v[k]=vec/c
    k+=1

In [154]:
x_test_w2v=np.zeros((len(df_test.Text),300))
k=0;
for i in df_test.Text:
    vec=np.zeros((300))
    c=1;
    for word in i.split(): 
        try:
            vec+=w2v[word]
            c+=1
        except:
            #print(word)
            pass
    x_test_w2v[k]=vec/c
    k+=1
    

In [159]:
model3=Sequential()
model3.add(InputLayer((300)))
model3.add(Dense(256))
model3.add(Dense(512))
model3.add(Dense(256,activation="relu"))
model3.add(Dense(128,activation="relu"))
model3.add(Dense(5,activation="softmax"))

In [160]:
model3.compile(loss="sparse_categorical_crossentropy",optimizer="adam",metrics="accuracy")

In [161]:
es=keras.callbacks.EarlyStopping(monitor='val_loss',patience=3,mode='min')

In [162]:
model3.fit(x_train_w2v,df_train.labels.values,epochs=100,batch_size=32,validation_split=0.1,callbacks=[es])

Epoch 1/100
42/42 [==============================] - 1s 9ms/step - loss: 0.7370 - accuracy: 0.7517 - val_loss: 0.3209 - val_accuracy: 0.8658
Epoch 2/100
42/42 [==============================] - 0s 4ms/step - loss: 0.1936 - accuracy: 0.9329 - val_loss: 0.1742 - val_accuracy: 0.9329
Epoch 3/100
42/42 [==============================] - 0s 3ms/step - loss: 0.1238 - accuracy: 0.9567 - val_loss: 0.1391 - val_accuracy: 0.9463
Epoch 4/100
42/42 [==============================] - 0s 4ms/step - loss: 0.0797 - accuracy: 0.9724 - val_loss: 0.1597 - val_accuracy: 0.9597
Epoch 5/100
42/42 [==============================] - 0s 4ms/step - loss: 0.0694 - accuracy: 0.9769 - val_loss: 0.1460 - val_accuracy: 0.9329
Epoch 6/100
42/42 [==============================] - 0s 4ms/step - loss: 0.0563 - accuracy: 0.9814 - val_loss: 0.1241 - val_accuracy: 0.9664
Epoch 7/100
42/42 [==============================] - 0s 4ms/step - loss: 0.0377 - accuracy: 0.9851 - val_loss: 0.1158 - val_accuracy: 0.9799
Epoch 8/100
4

In [164]:
model3.evaluate(x_test_w2v,df_labels.labels.values)

23/23 [==============================] - 0s 1ms/step - loss: 0.1433 - accuracy: 0.9646


[0.14329393208026886, 0.9646258354187012]

In [165]:
y_pred=np.argmax(model3.predict(x_test_w2v),axis=1)

In [166]:
pr=classification_report(df_labels.labels.values,y_pred)

In [167]:
print(pr)

              precision    recall  f1-score   support

           0       0.98      0.92      0.95       176
           1       0.99      0.96      0.98       135
           2       0.95      0.96      0.95       143
           3       0.99      0.99      0.99       168
           4       0.89      1.00      0.94       113

    accuracy                           0.96       735
   macro avg       0.96      0.97      0.96       735
weighted avg       0.97      0.96      0.96       735



In [170]:
f1_score(df_labels.labels.values,y_pred,average='macro')

0.9636016710110875